# Process INFORM Data Excel Files

This notebook processes all Excel files in the `inform_data` directory, extracts the "INFORM Severity - country" sheet from each, and combines them into CSV and parquet files.

In [17]:
import pandas as pd
import os
import re
from pathlib import Path

def extract_month_year(filename):
    """
    Extract month and year from filename in MONTH_YEAR format.
    Handles various filename patterns like:
    - 20221205_inform_severity_-_november_2022.xlsx -> november_2022
    - 202501_INFORM_Severity_-_January_2025.xlsx -> January_2025
    - 202511_inform_severity_-_late_november_2025_.xlsx -> november_2025
    """
    # Remove file extension
    name = filename.replace('.xlsx', '').replace('.XLSX', '')
    
    # Try to find pattern like "month_year" or "month_year_" at the end
    # Look for patterns like: word(s)_YYYY or YYYY_word(s)
    # Common pattern: something_month_year or something_YYYYMM
    
    # Pattern 1: Look for "month_year" pattern (e.g., "november_2022", "January_2025")
    # This matches: word(s) followed by underscore followed by 4-digit year
    pattern1 = r'([a-zA-Z]+(?:_[a-zA-Z]+)*)_(\d{4})(?:_|$)'
    match = re.search(pattern1, name, re.IGNORECASE)
    if match:
        month_part = match.group(1)
        year = match.group(2)
        # Convert month to lowercase for consistency, but keep original format
        month = month_part.lower()
        return month, year
    
    # Pattern 2: If filename starts with YYYYMM, extract from that
    # e.g., "202501_INFORM_Severity_-_January_2025" -> "January_2025"
    pattern2 = r'(\d{6})_.*?([a-zA-Z]+(?:_[a-zA-Z]+)*)_(\d{4})'
    match = re.search(pattern2, name, re.IGNORECASE)
    if match:
        month_part = match.group(2)
        year = match.group(3)
        month = month_part.lower()
        return month, year
    
    # Fallback: try to extract from any month_year pattern
    pattern3 = r'([a-zA-Z]+)_(\d{4})'
    matches = re.findall(pattern3, name, re.IGNORECASE)
    if matches:
        # Take the last match (most likely to be the month_year we want)
        month, year = matches[-1]
        return month.lower(), year
    
    # If no pattern matches, return a placeholder
    return "unknown_date", "unknown_year"

In [18]:
# Get the directory path (adjust if running from a different location)
inform_data_dir = Path("inform_data")

# Get all Excel files
excel_files = list(inform_data_dir.glob("*.xlsx")) + list(inform_data_dir.glob("*.XLSX"))

if not excel_files:
    print("No Excel files found in inform_data directory")
else:
    print(f"Found {len(excel_files)} Excel files to process")

Found 64 Excel files to process


In [19]:
dataframes = []

for excel_file in excel_files:
    try:
        print(f"Processing: {excel_file.name}")
        
        # Extract month_year from filename
        month, year = extract_month_year(excel_file.name)
        print(f"  Extracted date: {month} {year}")
        
        # Read the specific sheet
        # header=1 means use row 1 (second row, 0-indexed) as column names
        # This automatically skips row 0 (first row)
        df = pd.read_excel(excel_file, sheet_name="INFORM Severity - country", header=1)
        
        # Drop the first data row (which is row 2 in the original Excel file, the third row)
        # After using header=1, row 2 becomes index 0 in the dataframe
        if len(df) > 0:
            df = df.drop(df.index[0]).reset_index(drop=True)
            df = df.drop(df.index[0]).reset_index(drop=True)
        
        # Add month_year column
        df['month'] = month
        df['year'] = year
        
        dataframes.append(df)
        print(f"  Successfully loaded {len(df)} rows")
        
    except Exception as e:
        print(f"  Error processing {excel_file.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

Processing: 20230403_acaps_inform_severity_-_march_2023_1.xlsx
  Extracted date: march 2023
  Successfully loaded 82 rows
Processing: 20210706_inform_severity_-_june_2021.xlsx
  Extracted date: june 2021


/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Successfully loaded 74 rows
Processing: 202308_INFORM_Severity_-_August_2023.xlsx
  Extracted date: august 2023
  Successfully loaded 85 rows
Processing: 202401_INFORM_Severity_-_January_2024.xlsx
  Extracted date: january 2024


/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Successfully loaded 92 rows
Processing: 20210205_inform_severity_-_january_2021_0.xlsx
  Extracted date: january 2021
  Successfully loaded 73 rows
Processing: 20201105_inform_severity_-_october_2020.xlsx
  Extracted date: october 2020
  Successfully loaded 69 rows
Processing: 202512_inform_severity_mid_december_2025.xlsx
  Extracted date: inform_severity_mid_december 2025
  Successfully loaded 82 rows
Processing: 202312_INFORM_Severity_-_December_2023.xlsx
  Extracted date: december 2023
  Successfully loaded 91 rows
Processing: 202501_INFORM_Severity_-_January_2025.xlsx
  Extracted date: january 2025
  Successfully loaded 91 rows
Processing: 202505_INFORM_Severity_-_May_2025.xlsx
  Extracted date: may 2025
  Successfully loaded 87 rows
Processing: 20220606_inform_severity_-_may_2022.xlsx
  Extracted date: may 2022
  Successfully loaded 82 rows
Processing: 202404_INFORM_Severity_-_April_2024.xlsx
  Extracted date: april 2024


...

  Successfully loaded 72 rows
Processing: 20230803_INFORM_Severity_-_July_2023.xlsx
  Extracted date: july 2023
  Successfully loaded 84 rows
Processing: 20230202_inform_severity_-_january_2023_update.xlsx
  Extracted date: january 2023
  Successfully loaded 82 rows
Processing: 202309_INFORM_Severity_-_September_2023.xlsx
  Extracted date: september 2023
  Successfully loaded 84 rows
Processing: 20211101_inform_severity_-_october_2021.xlsx
  Extracted date: october 2021
  Successfully loaded 72 rows
Processing: 20220504_acaps_inform_severity_-_april_2022.xlsx
  Extracted date: april 2022
  Successfully loaded 82 rows
Processing: 202409_INFORM_Severity_-_September_2024.xlsx
  Extracted date: september 2024
  Successfully loaded 92 rows
Processing: 20230503_inform_severity_-_april_2023_0.xlsx
  Extracted date: april 2023
  Successfully loaded 84 rows
Processing: 20230302_inform_severity_-_february_2023.xlsx
  Extracted date: february 2023


/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Successfully loaded 83 rows
Processing: 20201021_inform_severity_-_september_2020.xlsx
  Extracted date: september 2020
  Successfully loaded 69 rows
Processing: 202503_INFORM_Severity_-_March_2025.xlsx
  Extracted date: march 2025
  Successfully loaded 89 rows
Processing: 202310_INFORM_Severity_-_October_2023.xlsx
  Extracted date: october 2023
  Successfully loaded 86 rows
Processing: 20220405_inform_severity_-_march_2022_0.xlsx
  Extracted date: march 2022
  Successfully loaded 82 rows
Processing: 20210401_inform_severity_-_march_2021.xlsx
  Extracted date: march 2021
  Successfully loaded 75 rows
Processing: 202506_INFORM_Severity_-_June_2025.xlsx
  Extracted date: june 2025


/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/Users/michaelperry/Desktop/code/datathon-2026/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'INFORM Severity - all crises'!$1:$67.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


  Successfully loaded 80 rows
Processing: 202502_INFORM_Severity_-_February_2025.xlsx
  Extracted date: february 2025
  Successfully loaded 90 rows


In [34]:
if not dataframes:
    print("No dataframes were successfully loaded")
else:
    # Concatenate all dataframes
    print("\nConcatenating all dataframes...")
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"Total rows in combined dataframe: {len(combined_df)}")
    print(f"Total columns: {len(combined_df.columns)}")
    
    # Display the dataframe
    display(combined_df)


Concatenating all dataframes...
Total rows in combined dataframe: 5292
Total columns: 24


,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,"Conflict,Violence,Displacement,Drought,Earthqu...",4.5,5,Very High,Stable,Very High,...,5,4,4.3,4,4.5,Asia,2023-03-27 00:00:00,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,Drought,3.1,4,High,Decreasing,High,...,3.7,4,2.2,3.1,1,Africa,2023-03-22 00:00:00,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,"Conflict,Displacement",1.1,2,Low,Decreasing,Medium,...,0.7,1,1.4,1.7,1,Middle east,2023-03-28 00:00:00,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,"Conflict,Displacement",1.8,2,Low,Stable,Low,...,0,2,2.3,2.5,2,Middle east,2023-03-28 00:00:00,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,"Violence,Displacement,Floods",3.5,4,High,Stable,Very High,...,3.8,3,3.3,3,3.5,Africa,2023-03-17 00:00:00,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,"Conflict,Displacement",4.4,5,Very High,Stable,Very High,...,5,4,4.1,2.4,5,Europe,2025-02-28 00:00:00,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,"Socio-political,Violence,Floods",4,4,High,Stable,Very High,...,4.8,3,3.7,3.3,4,Americas,2025-02-27 00:00:00,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,"Conflict,Displacement",4.5,5,Very High,Stable,Very High,...,5,4,4.5,3.8,5,Middle east,2025-02-10 00:00:00,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,"Drought,Food Security",3.5,4,High,Decreasing,High,...,5,3,2.2,2.4,2,Africa,2025-02-27 00:00:00,february,2025,NaN


In [35]:
# Clean data types before saving to parquet
# Convert numeric columns that may have mixed types (strings like 'x' mixed with numbers)
print("Cleaning data types for parquet compatibility...")
numeric_columns = ['INFORM Severity Index', 'INFORM Severity category', 
                'Conditions of people affected', 'People in need',
                   'Concentration of conditions', 'Complexity of the crisis', 
                   'Society and safety', 'Operating environment', 'Impact of the crisis']

# Date columns that should be converted to datetime
date_columns = ['Last updated']

# Convert DRIVERS from comma-separated string to list
if 'DRIVERS' in combined_df.columns:
    print("Converting DRIVERS from comma-separated to list format...")
    combined_df['DRIVERS'] = combined_df['DRIVERS'].apply(
        lambda x: [item.strip() for item in str(x).split(',')] if pd.notna(x) and str(x).strip() else []
    )

# Convert date columns first
for col in date_columns:
    if col in combined_df.columns:
        # Convert to datetime, replacing invalid dates with NaT
        combined_df[col] = pd.to_datetime(combined_df[col], errors='coerce')

# Also check for any object-type columns that might be numeric (but skip date columns)
for col in combined_df.columns:
    if col not in numeric_columns and col not in date_columns and combined_df[col].dtype == 'object':
        # Try to convert to numeric - if it works, it's likely a numeric column
        try:
            # Check if column contains mostly numeric values (excluding NaN)
            non_null = combined_df[col].dropna()
            if len(non_null) > 0:
                # Try converting a sample
                sample = non_null.head(100)
                numeric_count = pd.to_numeric(sample, errors='coerce').notna().sum()
                # If more than 50% can be converted to numeric, treat as numeric column
                if numeric_count / len(sample) > 0.5:
                    numeric_columns.append(col)
        except:
            pass

for col in numeric_columns:
    if col in combined_df.columns:
        # Convert to numeric, replacing non-numeric values with NaN
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

print(f"Cleaned {len(numeric_columns)} numeric columns")
print(f"Cleaned {len(date_columns)} date columns")
print("Data cleaning complete!")

Cleaning data types for parquet compatibility...
Cleaned 11 numeric columns
Cleaned 1 date columns
Data cleaning complete!


In [36]:
combined_df

,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,"Conflict,Violence,Displacement,Drought,Earthqu...",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.3,4.0,4.5,Asia,2023-03-27,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,Drought,3.1,4.0,High,Decreasing,High,...,3.7,4.0,2.2,3.1,1.0,Africa,2023-03-22,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,"Conflict,Displacement",1.1,2.0,Low,Decreasing,Medium,...,0.7,1.0,1.4,1.7,1.0,Middle east,2023-03-28,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,"Conflict,Displacement",1.8,2.0,Low,Stable,Low,...,0.0,2.0,2.3,2.5,2.0,Middle east,2023-03-28,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,"Violence,Displacement,Floods",3.5,4.0,High,Stable,Very High,...,3.8,3.0,3.3,3.0,3.5,Africa,2023-03-17,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,"Conflict,Displacement",4.4,5.0,Very High,Stable,Very High,...,5.0,4.0,4.1,2.4,5.0,Europe,2025-02-28,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,"Socio-political,Violence,Floods",4.0,4.0,High,Stable,Very High,...,4.8,3.0,3.7,3.3,4.0,Americas,2025-02-27,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,"Conflict,Displacement",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.5,3.8,5.0,Middle east,2025-02-10,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,"Drought,Food Security",3.5,4.0,High,Decreasing,High,...,5.0,3.0,2.2,2.4,2.0,Africa,2025-02-27,february,2025,NaN


In [37]:
# Save to CSV file
csv_path = Path("inform_severity_combined.csv")
print(f"Saving CSV to: {csv_path}")
combined_df.to_csv(csv_path, index=False)
print(f"Successfully saved CSV file with {len(combined_df)} rows")

# Save to parquet file
parquet_path = Path("inform_severity_combined.parquet")
print(f"\nSaving parquet to: {parquet_path}")
combined_df.to_parquet(parquet_path, index=False)
print(f"Successfully saved parquet file with {len(combined_df)} rows")

print(f"\n✓ Complete! Files saved:")
print(f"  CSV: {csv_path}")
print(f"  Parquet: {parquet_path}")

Saving CSV to: inform_severity_combined.csv
Successfully saved CSV file with 5292 rows

Saving parquet to: inform_severity_combined.parquet


: 

In [30]:
df = pd.read_parquet("inform_severity_combined.parquet")
df

,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,NaN,4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.3,4.0,4.5,Asia,2023-03-27,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,NaN,3.1,4.0,High,Decreasing,High,...,3.7,4.0,2.2,3.1,1.0,Africa,2023-03-22,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,NaN,1.1,2.0,Low,Decreasing,Medium,...,0.7,1.0,1.4,1.7,1.0,Middle east,2023-03-28,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,NaN,1.8,2.0,Low,Stable,Low,...,0.0,2.0,2.3,2.5,2.0,Middle east,2023-03-28,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,NaN,3.5,4.0,High,Stable,Very High,...,3.8,3.0,3.3,3.0,3.5,Africa,2023-03-17,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,NaN,4.4,5.0,Very High,Stable,Very High,...,5.0,4.0,4.1,2.4,5.0,Europe,2025-02-28,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,NaN,4.0,4.0,High,Stable,Very High,...,4.8,3.0,3.7,3.3,4.0,Americas,2025-02-27,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,NaN,4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.5,3.8,5.0,Middle east,2025-02-10,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,NaN,3.5,4.0,High,Decreasing,High,...,5.0,3.0,2.2,2.4,2.0,Africa,2025-02-27,february,2025,NaN
